In [14]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.1'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Fetched 252 kB in 2s (116 kB/

In [15]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-02-11 05:08:03--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.13MB/s    in 0.2s    

2021-02-11 05:08:03 (4.13 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [16]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("MultilingualJapanBigData").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [17]:
from pyspark import SparkFiles
# Load in user_data.csv from S3 into a DataFrame
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_multilingual_JP_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

multilingual_df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_multilingual_JP_v1_00.tsv.gz"), inferSchema=True, sep='\t', timestampFormat="mm/dd/yy")
multilingual_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------------------+----------------+-----------+-------------+-----------+----+-----------------+-------------------------------+-------------------------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|                   product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|                review_headline|                          review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------------------+----------------+-----------+-------------+-----------+----+-----------------+-------------------------------+-------------------------------------+-----------+
|         JP|      65317|R33RSUD4ZTRKT7|B000001GBJ|     957145596|            SONGS FROM A SECR...|           Music|          1|            1|         15|   N|                Y|                    残念ながら…| 残念ながら…趣味ではありません

In [18]:
# Count the number of the reviews 
print(multilingual_df.count())

262431


In [19]:
# Drop duplicates and incomplete rows 
dropped_df = multilingual_df.dropna()
print(dropped_df.count())

clean_multilingual_df = dropped_df.dropDuplicates()
print(clean_multilingual_df.count())

262422
262422


In [20]:
# Check the schema
clean_multilingual_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [21]:
# Create a new DataFrame for review info
review_id_table_jp = clean_multilingual_df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review_id_table_jp.show(5)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| RV04LCHIC0BHE|      73497|B00197U0DU|      68475645| 2013-07-09|
|R17093JI3IMACH|      94534|B00K0YXFK8|     827929624| 2014-06-29|
|R2920GFMZCE0VK|      95573|B004KKX814|     892527235| 2014-04-23|
|R1XIJILHR1JKG5|      98299|B006QJT442|     320190480| 2014-09-16|
|R3DWALNLTTC449|    6094456|B000EPFQ3M|     595283860| 2015-06-17|
+--------------+-----------+----------+--------------+-----------+
only showing top 5 rows



In [22]:
# Create a new DataFrame for unique values
products_jp = clean_multilingual_df.select(["product_id", "product_title"])
products_jp.show(5)

+----------+------------------------------------+
|product_id|                       product_title|
+----------+------------------------------------+
|B00197U0DU|                From Genesis to R...|
|B00K0YXFK8|ゴシップガール 〈コンプリート・シ...|
|B004KKX814|       ベイブレード BB-108 スター...|
|B006QJT442| ヘルボーイ ゴールデン・アーミー ...|
|B000EPFQ3M|          ティファニーで朝食を [DVD]|
+----------+------------------------------------+
only showing top 5 rows



In [23]:
# Total customers count 
customer_count = clean_multilingual_df.select(["customer_id"]).count()
customer_count

262422

In [24]:
# Create a new DataFrame for customers
customers_jp = clean_multilingual_df.groupBy('customer_id').count()
customers_jp_counts = customers_jp.withColumnRenamed('count', 'customers_count')
customers_jp_counts.show()

+-----------+---------------+
|customer_id|customers_count|
+-----------+---------------+
|   30354407|             46|
|   30792016|              1|
|   32982882|              1|
|   33060773|              2|
|   36603798|              2|
|   39960854|            230|
|   40606398|            115|
|   33745853|              3|
|   34176483|              1|
|   35396786|             18|
|   35754111|             27|
|   37893244|             31|
|   26880875|              1|
|   29760526|              5|
|   38572516|             10|
|   38619724|              4|
|   39283359|             10|
|   49053851|             53|
|    7249117|              1|
|   25500861|              2|
+-----------+---------------+
only showing top 20 rows



In [25]:
# vine table DataFrame
 
vine_jp = clean_multilingual_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_jp.show(5)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| RV04LCHIC0BHE|          1|            6|         28|   N|
|R17093JI3IMACH|          5|            4|          6|   N|
|R2920GFMZCE0VK|          5|            1|          1|   N|
|R1XIJILHR1JKG5|          5|            3|          4|   N|
|R3DWALNLTTC449|          5|            0|          1|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 5 rows



In [26]:
#@title config
# Write DataFrame to RDS
# Configuration for RDS instance

mode="append"
jdbc_url = "jdbc:postgresql://:5432/"
config = {"user": "",
          "password": "",
          "driver":"org.postgresql.Driver"}

In [27]:
# Write DataFrame to table

review_id_table_jp.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [28]:
# Write DataFrame to table

products_jp.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [29]:
# Write DataFrame to customers table in RDS

customers_jp_counts.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [30]:
# Write DataFrame to customers table in RDS

vine_jp.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)